In [10]:
# !export GOOGLE_APPLICATION_CREDENTIALS="/home/owang/.google/BigQuery.json"
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


### create dim_date table

In [6]:
%%bigquery --project bigquery-281204 --use_rest_api
create or replace table bigquery-281204.is6850.dim_date
    as (
        select 
            cast(format_date("%E4Y%m%d", dates) as numeric) as `DateKey`
            , format_date("%D", dates) as `Date`
            , extract(year from dates) as `Year`
            , extract(month from dates) as `Month`
            , extract(quarter from dates) as `Quarter`
            , extract(dayofweek from dates) as `DayOfWeek`
            , extract(isoweek from dates) as `ISOWeek`
            , format_date("%A", dates) as `DayName`
            , format_date("%B", dates) as `MonthName`
            , if(extract(dayofweek from dates)=1 or extract(dayofweek from dates)=7, 1, 0) as `IsWeekend`
        from unnest(generate_date_array('2010-01-01', '2030-01-01')) as `dates`
);

""


### create covid-19 summary table, including daily cases/death and accumulative cases/death

In [22]:
%%bigquery --project bigquery-281204 --use_rest_api
create or replace table bigquery-281204.is6850.covid19_summary as
    with temp as (
      select date
        , sum(confirmed_cases) as cumulative_confirmed_cases
        , sum(deaths) as cumulative_deaths
      from `bigquery-public-data.covid19_nyt.us_states`
      group by date
      order by date
    )
    select date
     , cumulative_confirmed_cases
     , cumulative_confirmed_cases - first_value(cumulative_confirmed_cases) over (diff) as daily_confirmed_cases
     , cumulative_deaths
     , cumulative_deaths - first_value(cumulative_deaths) over (diff) as daily_death
    from temp
    window diff as (
      order by date
      rows between 1 preceding and current row
    );

""


### create summary table include gas consumption and daily covid-19 cases, saved in array to preserve order

In [24]:
%%bigquery --project bigquery-281204 --use_rest_api
create or replace table bigquery-281204.is6850.summary_daily_cases as
  with temp as (
    select 
        date
        , daily_confirmed_cases
        , Weekly_US_Product_Supplied_of_Finished_Motor_Gasoline_Thousand_Barrels_per_Day as gas
    from bigquery-281204.is6850.covid19_summary dc
    join bigquery-281204.is6850.gas_consumption gc
    on dc.date = gc.week_of
  )
  select 
    array_agg(
      struct(date, daily_confirmed_cases, gas)
      order by date) 
      as summary
  from temp
;

""


### get standard devidation and mean then normalize data

In [26]:
%%bigquery --project bigquery-281204 --use_rest_api
create or replace table bigquery-281204.is6850.correlation_data as
with temp as (
  SELECT 
    date
    , daily_confirmed_cases
    , stddev(daily_confirmed_cases)
      over(statistics) as cases_stddev
    , avg(daily_confirmed_cases) 
      over(statistics) as cases_avg
    , gas
    , stddev(gas)
      over(statistics) as gas_stddev
    , avg(gas)
      over(statistics) as gas_avg
  FROM `bigquery-281204.is6850.summary_daily_cases`
  , unnest(`bigquery-281204.is6850.summary_daily_cases`.summary)
  window statistics as (
      order by date
      rows between unbounded preceding and unbounded following
  )
)
select
  date,
  (daily_confirmed_cases - cases_avg)/cases_stddev as norm_cases
  , (gas - gas_avg)/gas_stddev as norm_gas
from temp
;

""


### get Pearson Correlation coefficientcreate from above normalized data

In [29]:
%%bigquery --project bigquery-281204 --use_rest_api
SELECT round(corr(norm_cases, norm_gas), 4) as pearson
FROM `bigquery-281204.is6850.correlation_data`
;

,pearson
0,-0.3252
